Primero cargamos todas las librerias que usaremos,en conjunto con los PATHS de las bases

In [27]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
import statsmodels.api as sm
from statsmodels.compat import lzip
import datetime
from statsmodels.tools.tools import add_constant
import math
from scipy.stats import norm
from scipy.stats import chi2
#Cambiar el path *Importante
PATH_2210 = "E01_T_DEU_CONS_2210_REV.csv"
PATH_2211 = "E01_T_DEU_CONS_2211_REV.csv"
PATH_2212 = "E01_T_DEU_CONS_2212_REV.csv"
PATH_2301 = "E01_T_DEU_CONS_2301_REV.csv"
PATH_2302 = "E01_T_DEU_CONS_2302_REV.csv"
PATH_2303 = "E01_T_DEU_CONS_2303_REV.csv"
Lista_Path = [PATH_2210,PATH_2211,PATH_2212,PATH_2301,PATH_2302,PATH_2303]

In [8]:
def RutComunes(list):
    df2210 = pd.read_csv(PATH_2210, delimiter=";", usecols = ['RUT'])
    df2211 = pd.read_csv(PATH_2211, delimiter=";", usecols = ['RUT'])
    df2212 = pd.read_csv(PATH_2212, delimiter=";", usecols = ['RUT'])
    df2301 = pd.read_csv(PATH_2301, delimiter=";", usecols = ['RUT'])
    df2302 = pd.read_csv(PATH_2302, delimiter=";", usecols = ['RUT'])
    df2303 = pd.read_csv(PATH_2303, delimiter=";", usecols = ['RUT'])
    RutComunes = np.intersect1d(np.intersect1d(np.intersect1d(df2210, df2211), df2212), np.intersect1d(df2301, np.intersect1d(df2302, df2303)))
    dfRutComunes = pd.DataFrame({'RUT': RutComunes})
    lista_rut = dfRutComunes["RUT"]
    return lista_rut

RutComunes(Lista_Path)

0                0
1                1
2                3
3                4
4                6
            ...   
1028022    1083121
1028023    1083122
1028024    1083123
1028025    1083124
1028026    1083125
Name: RUT, Length: 1028027, dtype: int64

Generamos el Test de Durbin Watson y el de tamaño muestra

In [58]:
def DataProm(dataframe):
    cantidad_filas = dataframe.shape[0]
    tamaño_muestra =calcular_tamano_muestra(cantidad_filas,0.05,0.5,0.05)
    lista_dataframes =[]
    for i in range (31):
        numero_aleatorio = random.randint(tamano_muestra,cantidad_filas-tamaño_muestra)
        dfparalista = dataframe.iloc[numero_aleatorio-1:numero_aleatorio+tamaño_muestra]
        dfparalista.columns = dataframe.columns
        lista_dataframes.append(dfparalista)
    
    #print(lista_dataframes[30])
    num_dataframes = len(lista_dataframes)
    num_filas = lista_dataframes[0].shape[0]
    num_columnas = lista_dataframes[0].shape[1]
    df_promedio = pd.DataFrame(np.zeros((num_filas, num_columnas)))
    for i in range(num_filas):
        for j in range(num_columnas):
            suma = sum(df.iloc[i, j] for df in lista_dataframes)
            promedio = suma / num_dataframes
            df_promedio.iloc[i, j] = promedio
    df_promedio = df_promedio.drop(columns=[6])
    df_promedio = df_promedio.rename(columns={0: '2210', 1: '2211', 2: '2212', 3: '2301', 4: '2302', 5: '2303'})
    return df_promedio



def TestSeriesDW (dataframe,fecha_1,fecha_2):

    df_fecha_1 = dataframe[str(fecha_1)]
    df_fecha_2 = dataframe[str(fecha_2)]
    dw_test = durbin_watson(df_fecha_2 - df_fecha_1)
    print(f"El test de DW entre las fechas de {str(fecha_1)} y {str(fecha_2)} es de :{dw_test}")


def calcular_tamano_muestra(N, alpha, p, d):
    z = abs(norm.ppf(1 - alpha/2)) 
    numerator = (N * z**2 * p * (1 - p))
    denominator = (d**2 * (N - 1) + z**2 * p * (1 - p))
    n = numerator / denominator
    muestra_tamaño = math.ceil(n)
    return muestra_tamaño
N = 1000000
alpha = 0.05
p = 0.5
d = 0.05

tamano_muestra = calcular_tamano_muestra(N, alpha, p, d)
print("Tamaño de la muestra:", tamano_muestra)

Tamaño de la muestra: 384


In [59]:
def Test(Variable):
    df2210 = pd.read_csv(PATH_2210, delimiter=";", usecols = ['RUT'])
    df2211 = pd.read_csv(PATH_2211, delimiter=";", usecols = ['RUT'])
    df2212 = pd.read_csv(PATH_2212, delimiter=";", usecols = ['RUT'])
    df2301 = pd.read_csv(PATH_2301, delimiter=";", usecols = ['RUT'])
    df2302 = pd.read_csv(PATH_2302, delimiter=";", usecols = ['RUT'])
    df2303 = pd.read_csv(PATH_2303, delimiter=";", usecols = ['RUT'])
    RutComunes = np.intersect1d(np.intersect1d(np.intersect1d(df2210, df2211), df2212), np.intersect1d(df2301, np.intersect1d(df2302, df2303)))
    dfRutComunes = pd.DataFrame({'RUT': RutComunes})
    lista_rut = dfRutComunes["RUT"]
    ###########################################################################
    df22101 = pd.read_csv(PATH_2210, delimiter=";", usecols = ['RUT',Variable])
    df22111 = pd.read_csv(PATH_2211, delimiter=";", usecols = ['RUT',Variable])
    df22121 = pd.read_csv(PATH_2212, delimiter=";", usecols = ['RUT',Variable])
    df23011 = pd.read_csv(PATH_2301, delimiter=";", usecols = ['RUT',Variable])
    df23021 = pd.read_csv(PATH_2302, delimiter=";", usecols = ['RUT',Variable])
    df23031 = pd.read_csv(PATH_2303, delimiter=";", usecols = ['RUT',Variable])
    dfvarible22101 = df22101.loc[df22101['RUT'].isin(lista_rut)].dropna()
    dfvarible22111 = df22111.loc[df22111['RUT'].isin(lista_rut)].dropna()
    dfvarible22121 = df22121.loc[df22121['RUT'].isin(lista_rut)].dropna()
    dfvarible23011 = df23011.loc[df23011['RUT'].isin(lista_rut)].dropna()
    dfvarible23021 = df22101.loc[df23021['RUT'].isin(lista_rut)].dropna()
    dfvarible23031 = df23031.loc[df23031['RUT'].isin(lista_rut)].dropna()

    dataframes = [
        (dfvarible22101, '2210'),
        (dfvarible22111, '2211'),
        (dfvarible22121, '2212'),
        (dfvarible23011, '2301'),
        (dfvarible23021, '2302'),
        (dfvarible23031, '2303')
    ]

    dftemporal = pd.DataFrame()

    for df, nombre_df in dataframes:
        dftemporal[nombre_df] = df[Variable]

    dftemporal['RUT'] = df22101['RUT'] 
    dftemporal = dftemporal.dropna()
    dataframeDW = DataProm(dftemporal)
    TestSeriesDW(dataframeDW,2210,2211)
    TestSeriesDW(dataframeDW,2211,2212)
    TestSeriesDW(dataframeDW,2212,2301)
    TestSeriesDW(dataframeDW,2301,2302)
    TestSeriesDW(dataframeDW,2302,2303)

In [37]:
Test("SPROM") #14 o 22

             2210       2211         2212       2301      2302        2303  \
1735350    5747.0  1166577.0          0.0  3832855.0    5747.0    157835.0   
1735351    9640.0   991315.0          0.0        0.0    9640.0   2047287.0   
1736359       0.0  2691938.0          0.0   385827.0       0.0  10875158.0   
1736871       0.0    28034.0  535295008.0        0.0       0.0   1559569.0   
1737943    1282.0  1445314.0          0.0        0.0    1282.0         0.0   
...           ...        ...          ...        ...       ...         ...   
1879383   34403.0   219558.0    1423598.0        0.0   34403.0    854160.0   
1880845  406068.0   183238.0   22675458.0        0.0  406068.0         0.0   
1880890  883283.0        0.0          0.0        0.0  883283.0         0.0   
1881475       0.0        0.0          0.0  1235532.0       0.0      1531.0   
1881720       0.0        0.0          0.0        0.0       0.0         0.0   

            RUT  
1735350  533146  
1735351  533147  
1736359  

In [40]:
Test("SPROM")#24

              2210      2211      2212       2301       2302      2303     RUT
1022430        0.0       0.0       0.0   391643.0        0.0       0.0  760657
1022431        0.0       0.0       0.0     3180.0        0.0       0.0  760656
1024747        0.0       0.0       0.0      523.0        0.0       0.0  886845
1024748        0.0       0.0       0.0  3005009.0        0.0       0.0  886844
1024750        0.0   98726.0       0.0        0.0        0.0       0.0  886836
...            ...       ...       ...        ...        ...       ...     ...
1182037        0.0       0.0       0.0  1256157.0        0.0       0.0  764237
1182310   720358.0  532449.0   55360.0        0.0   720358.0       0.0  730151
1182311       65.0  167568.0       0.0        0.0       65.0  860975.0  730146
1182312  2762371.0       0.0  443600.0        0.0  2762371.0   10071.0  730145
1182313        0.0       0.0    3281.0        0.0        0.0   23231.0  730157

[362 rows x 7 columns]


In [42]:
Test("SPROM")#30

               2210  2211       2212  2301        2302  2303     RUT
1652428      7595.0   0.0        0.0   0.0      7595.0   0.0  366486
1652429  27064562.0   0.0        0.0   0.0  27064562.0   0.0  366487
1652430   3765540.0   0.0        0.0   0.0   3765540.0   0.0  365904
1652431    164538.0   0.0        0.0   0.0    164538.0   0.0  365900
1652432     27266.0   0.0        0.0   0.0     27266.0   0.0  365895
...             ...   ...        ...   ...         ...   ...     ...
1832730         0.0   0.0   792302.0   0.0         0.0   0.0  854117
1832731         0.0   0.0  1065102.0   0.0         0.0   0.0  854117
1832733         0.0   0.0    15491.0   0.0         0.0   0.0  854115
1832734         0.0   0.0        0.0   0.0         0.0   0.0  854123
1833232         0.0   0.0    43581.0   0.0         0.0   0.0  862802

[362 rows x 7 columns]


Hasta aca el codigo solo llegaba al print de lista_dataframe[posicion], despues se le agrega lo del promedio y se muestra el dataframe del promedio en el mismo formato

In [55]:
Test("SPROM")#El Promedio

             2210          2211          2212          2301          2302  \
0    4.064911e+05  7.317766e+05  6.755112e+05  1.023579e+06  4.064911e+05   
1    1.487941e+06  5.998129e+05  5.889349e+05  1.015591e+06  1.487941e+06   
2    7.434383e+06  2.092365e+06  2.597273e+06  2.096709e+06  7.434383e+06   
3    2.423483e+06  8.118126e+04  1.291432e+06  2.897727e+06  2.423483e+06   
4    1.483898e+07  4.121446e+05  3.048376e+06  3.167792e+05  1.483898e+07   
..            ...           ...           ...           ...           ...   
357  9.039152e+05  1.022790e+07  3.166059e+07  2.760176e+06  9.039152e+05   
358  9.759952e+05  2.788456e+06  2.695562e+06  5.959854e+05  9.759952e+05   
359  1.346534e+06  2.972924e+06  4.373158e+06  1.341053e+05  1.346534e+06   
360  1.581608e+06  1.067521e+06  4.852287e+05  5.583000e+06  1.581608e+06   
361  2.936192e+06  9.391189e+05  4.471568e+05  7.596864e+05  2.936192e+06   

             2303  
0    1.328041e+06  
1    1.950078e+06  
2    3.797017e+

Finalmente se llega a calcular Dw segun las fechas

In [61]:
Test("MTOINT")

El test de DW entre las fechas de 2210 y 2211 es de :1.5958038122121412
El test de DW entre las fechas de 2211 y 2212 es de :1.4826854812251307
El test de DW entre las fechas de 2212 y 2301 es de :1.3017888585258857
El test de DW entre las fechas de 2301 y 2302 es de :1.4420626073425113
El test de DW entre las fechas de 2302 y 2303 es de :1.7241460934995478
